In [1]:
!pip install kagglehub

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)


In [2]:
import kagglehub
# Download latest version
path = kagglehub.dataset_download("clmentbisaillon/fake-and-real-news-dataset")

print("Path to dataset files:", path)

Path to dataset files: /home/ezka/.cache/kagglehub/datasets/clmentbisaillon/fake-and-real-news-dataset/versions/1


In [3]:
from paths import DATA_DIR
import shutil

shutil.move(str(path), str(DATA_DIR))

'/mnt/c/Users/atrol/PycharmProjects/Fake-News/data/1'